# Vehicle Choice Mixed Logit

In [ ]:
import pandas as pd

import larch as lx
from larch import PX

In this example, we will demonstrate the estimatation of parameters for
mixed logit models.  The examples are based on a problem set originally published by Ken Train.

To ensure good preformance and ease of use, estimating parameters for a mixed logit model in Larch requires the `jax` library as the compute engine.  Importing it here allows us to confirm that it is available.

In [ ]:
import jax  # noqa: F401

The data represent consumers' choices among vehicles in stated preference experiments. The data are from a study that he did for Toyota and GM to assist them in their analysis of the potential marketability of electric and hybrid vehicles, back before hybrids were introduced.   

In each choice experiment, the respondent was presented with three vehicles, with the price and other attributes of each vehicle described. The respondent was asked to state which of the three vehicles he/she would buy if the these vehicles were the only ones available in the market. There are 100 respondents in our dataset (which, to reduce estimation time, is a subset of the full dataset which contains 500 respondents.) Each respondent was presented with 15 choice experiments, and most respondents answered all 15. The attributes of the vehicles were varied over experiments, both for a given respondent and over respondents. The attributes are: price, operating cost in dollars per month, engine type (gas, electric, or hybrid), range if electric (in hundreds of miles between recharging), and the performance level of the vehicle (high, medium, or low). The performance level was described in terms of top speed and acceleration, and these descriptions did not vary for each level; for example, "High" performance was described as having a top speed of 100 mpg and 12 seconds to reach 60 mpg, and this description was the same for all "high" performance vehicles. 

In [ ]:
raw_data = pd.read_csv(
    "data.txt",
    names=[
        "person_id",
        "case_id",
        "chosen",
        "price",  # dollars
        "opcost",  # dollars per month
        "max_range",  # hundreds of miles (0 if not electric)
        "ev",  # bool
        "gas",  # bool
        "hybrid",  # bool
        "hiperf",  # High performance (bool)
        "medhiperf",  # Medium or high performance (bool)
    ],
    sep=r"\s+",
)

In [ ]:
raw_data

Create `alt_id` values for each row within each group, and set
the index on case and alternative id's.

In [ ]:
raw_data["alt_id"] = raw_data.groupby("case_id").cumcount() + 1
raw_data = raw_data.set_index(["case_id", "alt_id"])

In [ ]:
data = lx.Dataset.construct.from_idca(raw_data)

In [ ]:
data

## Simple MNL

Start with a simple MNL model.

In [ ]:
simple = lx.Model(data)

In [ ]:
simple.utility_ca = (
    PX("price")
    + PX("opcost")
    + PX("max_range")
    + PX("ev")
    + PX("hybrid")
    + PX("hiperf")
    + PX("medhiperf")
)

In [ ]:
simple.choice_ca_var = "chosen"

In [ ]:
simple.maximize_loglike(stderr=True, options={"ftol": 1e-9});

In [ ]:
# TEST
from pytest import approx

assert simple.most_recent_estimation_result["loglike"] == approx(-1399.1932)

expected_value = {
    "price": -0.4167 / 10000,
    "opcost": -0.1288 / 10,
    "max_range": 0.4770,
    "ev": -1.3924,
    "hybrid": 0.3555,
    "hiperf": 0.1099,
    "medhiperf": 0.3841,
}
assert simple.parameters["value"].to_series().to_dict() == approx(
    expected_value, rel=1e-3
)

expected_stderr = {
    "price": 0.0332 / 10000,
    "opcost": 0.0353 / 10,
    "max_range": 0.1765,
    "ev": 0.2766,
    "hybrid": 0.1218,
    "hiperf": 0.0838,
    "medhiperf": 0.0855,
}
assert simple.parameters["std_err"].to_series().to_dict() == approx(
    expected_stderr, rel=2e-3
)

## Mixture Model

To create a mixed logit model, we can start with a copy of the simple model.

In [ ]:
mixed = simple.copy()

We assign mixtures to various parameters by providing a list of `Mixture` definitions.

In [ ]:
mixed.mixtures = [
    lx.mixtures.Normal(k, f"s_{k}")
    for k in ["opcost", "max_range", "ev", "hybrid", "hiperf", "medhiperf"]
]

The estimation of the mixed logit parameters sometimes performs
poorly (i.e. may prematurely call itself converged) if you start
from the MNL solution, so it can be advantageous to start from 
the null parameters instead.

In [ ]:
mixed.pvals = "null"

There are several extra settings that can be manipulated on the
mixed logit model, including the number of draws and a random seed
for generating draws.

In [ ]:
mixed.n_draws = 200
mixed.seed = 42

In [ ]:
mixed.maximize_loglike(stderr=True, options={"ftol": 1e-9});

In [ ]:
mixed.parameters["std_err"].to_series().to_dict()

In [ ]:
# TEST
assert mixed.most_recent_estimation_result["loglike"] == approx(-1385.4436)

expected_value = {
    "ev": -3.116113749980346,
    "hiperf": 0.16526011648413752,
    "hybrid": 0.5468359000231515,
    "max_range": 0.893907686323169,
    "medhiperf": 0.7444412811439853,
    "opcost": -0.022305055541141636,
    "price": -6.53728508122224e-05,
    "s_ev": 2.8852968025136363,
    "s_hiperf": 0.7816346849525359,
    "s_hybrid": -1.2110724172285334,
    "s_max_range": -0.10080141887433067,
    "s_medhiperf": -1.5364071517526834,
    "s_opcost": 0.04830038015701635,
}
assert mixed.parameters["value"].to_series().to_dict() == approx(
    expected_value, rel=1e-3
)

expected_stderr = {
    "ev": 0.8305132389068604,
    "hiperf": 0.12845700979232788,
    "hybrid": 0.20117034018039703,
    "max_range": 0.33233582973480225,
    "medhiperf": 0.21790540218353271,
    "opcost": 0.007205564994364977,
    "price": 1.0975929399137385e-05,
    "s_ev": 0.7865121364593506,
    "s_hiperf": 0.8348018527030945,
    "s_hybrid": 0.6305373907089233,
    "s_max_range": 0.9989855885505676,
    "s_medhiperf": 0.6614137887954712,
    "s_opcost": 0.0211756881326437,
}
assert mixed.parameters["std_err"].to_series().to_dict() == approx(
    expected_stderr, rel=2e-3
)

## Panel Data

In the mixed logit model above, we have ignored the fact that 
we have "panel data", where multiple observations are made by the
same decision maker who (presumably) has stable preferences with
respect to the choice attributes.  We can tell Larch which data
column represents the panel identifiers (`person_id`) and then
use that information to build a significantly better model.

In [ ]:
panel = mixed.copy()

In [ ]:
panel.groupid = "person_id"

In [ ]:
panel.seed = 42

In [ ]:
panel.maximize_loglike(stderr=True, options={"ftol": 1e-9});

In [ ]:
panel.parameters["std_err"].to_series().to_dict()

In [ ]:
# TEST
assert panel.most_recent_estimation_result["loglike"] == approx(-1334.9873046875)

expected_value = {
    "ev": -1.7534512916797778,
    "hiperf": 0.1058554151399541,
    "hybrid": 0.4557416859213061,
    "max_range": 0.5651797656380667,
    "medhiperf": 0.5342572130829607,
    "opcost": -0.014340199116707007,
    "price": -4.905225237322768e-05,
    "s_ev": -1.066473079936292,
    "s_hiperf": 0.35845535684013297,
    "s_hybrid": -0.8534250562601524,
    "s_max_range": 0.4090876192252884,
    "s_medhiperf": -0.5090464136312128,
    "s_opcost": 0.03259632742913284,
}
assert panel.parameters["value"].to_series().to_dict() == approx(
    expected_value, rel=1e-3
)

expected_stderr = {
    "ev": 0.3464612364768982,
    "hiperf": 0.10119915008544922,
    "hybrid": 0.1610344499349594,
    "max_range": 0.21604567766189575,
    "medhiperf": 0.11232057213783264,
    "opcost": 0.00524064339697361,
    "price": 3.8892262637091335e-06,
    "s_ev": 0.22575485706329346,
    "s_hiperf": 0.18787971138954163,
    "s_hybrid": 0.15704098343849182,
    "s_max_range": 0.22642944753170013,
    "s_medhiperf": 0.16932818293571472,
    "s_opcost": 0.005151339806616306,
}
assert panel.parameters["std_err"].to_series().to_dict() == approx(
    expected_stderr, rel=2e-3
)